In [4]:
model_name = 'Helsinki-NLP/opus-mt-ny-en'


In [5]:
from transformers import MarianMTModel, MarianTokenizer


In [6]:
from transformers import MarianTokenizer, MarianMTModel
import tensorflow as tf

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")
outputs = model(inputs)

last_hidden_states = outputs.last_hidden_state

AttributeError: 

In [8]:
import pandas as pd

In [118]:
TRAIN_FILEPATH = "../Translated/cleaned/train.csv"
TEST_FILEPATH = "../Translated/cleaned/test.csv"
SS_FILEPATH = "../data/SampleSubmission.csv"
VECTORS_FILEPATH = ""
train = pd.read_csv(TRAIN_FILEPATH)
test = pd.read_csv(TEST_FILEPATH)
ss = pd.read_csv(SS_FILEPATH)

In [119]:
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name, output_loading_info=False)
        
# batch2 = tokenizer.prepare_seq2seq_batch(train.Text.values.tolist()[0],
#                                                max_length=500,
#                                                pad_to_max_length=True,
#                                        return_tensors = 'pt')
# translated = model.generate(**batch)

# df.loc[df['lang']==lang, 'content_english'] = 

In [133]:
# batch['input_ids']
# import numpy as np
# (np.array((tokenizer.encode(train.Text.values.tolist()[0],max_length = 500)))==batch['input_ids'][0].numpy()).all()
# model.model.encoder.layers[:6]
# layer1 = model.model.encoder.layers.state_dict()
tokens = []
for text in train.Text.values.tolist() : 
    tokens.append(np.array((tokenizer.encode(text,max_length = 500,padding = 'max_length',truncation = True))))

In [142]:
# type(layer1)
# train['tokens'] = tokens
# train.tokens.apply(len)
# (tokenizer.encode.__doc__)
# ((tokenizer.encode(train.Text.values.tolist(),max_length = 500,padding = 'max_length',truncation = True)))
# train
import tensorflow as tf

train_text = tf.keras.preprocessing.sequence.pad_sequences(
    train.tokens, maxlen=500, dtype='int32', padding='post',
    truncating='post'
)
test_text = tf.keras.preprocessing.sequence.pad_sequences(
    train.tokens, maxlen=500, dtype='int32', padding='post',
    truncating='post'
)

In [143]:
# # **batch
# # output = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
# # type(translated)
# # batch.keys()
# # batch['input_ids'][0]
# # train.Text.str.split(' ')[0][6]
# # batch2 = tokenizer.prepare_seq2seq_batch(train.Text.values.tolist()[1],
# #                                                max_length=500,
# #                                                pad_to_max_length=True,
# #                                        return_tensors = 'pt')
# # batch2
# # train.Text.values.tolist()
# test_tokens = []

# for text in test.Text.values.tolist() : 
#     test_tokens.append(np.array((tokenizer.encode(text,max_length = 500,padding = True))))


In [96]:
# train.Text.values.tolist()[1]
test['tokens'] = test_tokens
# output
# translated.shape
# from transformers import pipeline
# pipeline('zero-shot-classification', model=model_name, tokenizer=tokenizer)

In [98]:
test.drop(['Text'],axis = 1, inplace = True)
train.drop(['Text'],axis = 1, inplace = True)

In [145]:
# len(train.Text[0].split())
# model.get_encoder().tie_weights()
# test
# train
# cols_target = train.Label.unique().tolist()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_train = lb.fit_transform(train['Label'])

y_train = pd.DataFrame(y_train, columns= lb.classes_)
# y_train
cols_target = train['Label'].unique().tolist()
train_df = pd.concat([train, y_train], axis = 1)

x_train, x_val, y_train, y_val = train_test_split(train_text, train_df[cols_target], test_size=0.1, random_state = 0,stratify = train_df['Label'])


In [147]:
# train
x_train.shape

(1292, 500)

In [148]:
# x_train.apply(len)

In [168]:
# dir(model.train())
# type(model.get_input_embeddings())
# train_df
# x_train.shape
from tensorflow import keras
layers = keras.layers
models = keras.models
# Build the model
from keras import backend as K 

# Do some code, e.g. train and save model

K.clear_session()
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
num_filters = 7
model = models.Sequential()
model.add(layers.Embedding(60000, 300, input_length=500, trainable=True))
model.add(layers.Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(40, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.5)))
# model.add(Dense(num_classes, activation='sigmoid'))  #multi-label (k-hot encoding)
# base_model.add(layers.Dropout(0.2))
# model.add(layers.Dense(2048))
# model.add(layers.BatchNormalization())
# model.add(layers.Activation('relu'))
# model.add(layers.Dense(512))
# # model.add(layers.BatchNormalization())
# model.add(layers.Activation('relu'))
# model.add(layers.Dense(128))
# # model.add(layers.BatchNormalization())
# model.add(layers.Activation('relu'))

# model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(20))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [169]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=50,
                    verbose=1,
                   validation_split = 0.1)

Epoch 1/50
19/19 [==============================] - 9s 474ms/step - loss: 8.8968 - accuracy: 0.1274 - val_loss: 8.3767 - val_accuracy: 0.2231
Epoch 2/50
19/19 [==============================] - 9s 465ms/step - loss: 7.9429 - accuracy: 0.2014 - val_loss: 7.4946 - val_accuracy: 0.2077
Epoch 3/50
19/19 [==============================] - 9s 464ms/step - loss: 7.0624 - accuracy: 0.1773 - val_loss: 6.6769 - val_accuracy: 0.2231
Epoch 4/50
19/19 [==============================] - 10s 505ms/step - loss: 6.2628 - accuracy: 0.1902 - val_loss: 5.9622 - val_accuracy: 0.2154
Epoch 5/50
19/19 [==============================] - 9s 492ms/step - loss: 5.6114 - accuracy: 0.1988 - val_loss: 5.3806 - val_accuracy: 0.1923
Epoch 6/50
19/19 [==============================] - 9s 493ms/step - loss: 5.0216 - accuracy: 0.2384 - val_loss: 4.9016 - val_accuracy: 0.2077
Epoch 7/50
19/19 [==============================] - 9s 491ms/step - loss: 4.5601 - accuracy: 0.2427 - val_loss: 4.4859 - val_accuracy: 0.2077
Epoch

In [165]:
test_text.max()

59678